In [6]:
from PIL import Image
import numpy as np
import torch
import os
from transformers import DPTForDepthEstimation, DPTFeatureExtractor

model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas", low_cpu_mem_usage=True)
feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-hybrid-midas")

# Path to the folder containing RGB images
input_folder = '/Users/muhammadalimiran/Desktop/Livenesss Detection/real_augmented'

# Path to the output folder for depth maps
output_folder = '/Users/muhammadalimiran/Desktop/Livenesss Detection/real_depthmap'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path)
        
        # Prepare image for the model
        inputs = feature_extractor(images=image, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)
            predicted_depth = outputs.predicted_depth

        # Interpolate to original size
        prediction = torch.nn.functional.interpolate(
            predicted_depth.unsqueeze(1),
            size=image.size[::-1],
            mode="bicubic",
            align_corners=False,
        )

        # Convert the depth prediction to a numpy array
        depth_map = prediction.squeeze().cpu().numpy()

        # Normalize depth values for visualization
        formatted = (depth_map * 255 / np.max(depth_map)).astype("uint8")

        # Create a depth map image
        depth_image = Image.fromarray(formatted)

        # Save the depth map image in the output folder
        output_path = os.path.join(output_folder, filename.replace('.jpg', '_depth.jpg'))
        depth_image.save(output_path)

print("Depth maps generated and saved in", output_folder)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e32672fb-0a45-4109-b596-76e9cc73e028)')' thrown while requesting HEAD https://huggingface.co/Intel/dpt-hybrid-midas/resolve/main/config.json


KeyboardInterrupt: 